In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

## Data

### Tickers

In [3]:
df_tickers = pd.read_csv('../data/etf_list.csv').replace('BTC-USD', 'BTC', regex=True)
df_tickers.set_index('ticker', inplace=True)
df_tickers

,name,full
ticker,,
BTC,Bitcoin,Bitcoin (BTC)
QQQ,US Nasdaq 100,US Nasdaq 100 (QQQ)
...,...,...
BIL,US Cash,US Cash (BIL)
DBC,Commodities,Commodities (DBC)


In [4]:
df_tickers.index

Index(['BTC', 'QQQ', 'IWF', 'SPY', 'MDY', 'IWD', 'IWM', 'CWB', 'VNQ', 'EFA',
       'HYG', 'PFF', 'LQD', 'EMB', 'GLD', 'TLT', 'TIP', 'BND', 'EEM', 'BIL',
       'DBC'],
      dtype='object', name='ticker')

### Historical Data

In [5]:
df_hist = pd.read_csv('../data/etf_data.csv', index_col=0, parse_dates=True, header=[0, 1])
df_hist.rename({'BTC-USD': 'BTC'}, axis=1, level=1, inplace=True)
df_hist

Adj Close                                                ...  \
                 BIL BND           BTC CWB DBC EEM EFA EMB GLD HYG  ...   
Date                                                                ...   
1993-01-29       NaN NaN           NaN NaN NaN NaN NaN NaN NaN NaN  ...   
1993-02-01       NaN NaN           NaN NaN NaN NaN NaN NaN NaN NaN  ...   
...              ...  ..           ...  ..  ..  ..  ..  ..  ..  ..  ...   
2024-04-06       NaN NaN  68896.109375 NaN NaN NaN NaN NaN NaN NaN  ...   
2024-04-07       NaN NaN  69352.804688 NaN NaN NaN NaN NaN NaN NaN  ...   

           Volume                                             
              IWF IWM LQD MDY PFF QQQ        SPY TIP TLT VNQ  
Date                                                          
1993-01-29    NaN NaN NaN NaN NaN NaN  1003200.0 NaN NaN NaN  
1993-02-01    NaN NaN NaN NaN NaN NaN   480500.0 NaN NaN NaN  
...           ...  ..  ..  ..  ..  ..        ...  ..  ..  ..  
2024-04-06    NaN NaN NaN NaN NaN NaN        NaN NaN NaN NaN  
2024-04-07    NaN NaN NaN NaN NaN NaN        NaN NaN NaN NaN  

[8939 rows x 126 columns]

In [7]:
df_hist['Adj Close'].to_parquet('../../../multiple_assets.parquet')

In [60]:
from mplcustom import datastyle as ds

In [61]:
ds.save_df(df_hist, 'hist', max_rows=8, max_cols=8)

## Methodology

### Cumulative return yearly

In [62]:
df = df_hist.loc['2010':, 'Adj Close']

df = (df
 .groupby(df.index.year).pct_change().add(1)
 .groupby(df.index.year).cumprod().sub(1)
 .resample('YE').last().T
)

/var/folders/xm/_8bv9n2n5f3f64xxt_d5_l880000gn/T/ipykernel_24195/339510106.py:4: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .groupby(df.index.year).pct_change().add(1)


In [63]:
df.columns = df.columns.year

In [64]:
from mplcustom import datastyle as ds

In [65]:
ds.save_df(df, 'etf_returns')

### Row summary

In [66]:
ta = df.mean(axis=1).mul(100)
tc = df.add(1).cumprod(axis=1).sub(1).mul(100).iloc[:,[-1]]

t = pd.concat([ta, tc], axis=1)
t.columns = ['AVG', 'CUM']

In [67]:
ds.save_df(t, 'etf_summary', max_rows=6)

In [68]:
dfr = pd.concat({
    'YEARLY': df.mul(100),
    'TOTAL': t
}, axis=1)

dfr

YEARLY                                                         \
             2010       2011       2012       2013       2014       2015   
BIL  6.661338e-13  -0.061076  -0.021803  -0.087317  -0.065566  -0.109297   
BND  6.077207e+00   8.064261   3.262274  -1.955761   5.725347   0.218439   
BTC           NaN        NaN        NaN        NaN -29.987059  37.014595   
CWB  1.280418e+01  -7.975849  13.982726  19.287528   8.005454  -0.838895   
DBC  9.152138e+00  -2.999639   0.506525  -7.962695 -27.046266 -26.714207   
EEM  1.322195e+01 -19.570260  15.517695  -5.502211  -0.048793 -15.050538   
EFA  5.404694e+00 -12.947910  15.608950  19.593440  -4.500392  -0.568004   
EMB  1.067502e+01   7.198319  17.603877  -8.047667   6.025027   1.395162   
GLD  2.633880e+01  10.137685   3.912267 -28.834955  -3.745761 -11.062415   
HYG  1.104099e+01   6.671975  10.696739   4.765883   1.719011  -5.021535   
IWD  1.326878e+01  -0.983614  15.552523  28.937020  14.254313  -3.999696   
IWF  1.500921e+01   1.383875  13.562420  29.754817  13.778403   5.590898   
IWM  2.386339e+01  -6.059659  14.804630  34.733873   6.186893  -3.924730   
LQD  8.742207e+00   9.317315  10.331629  -2.227873   8.001854  -1.601613   
MDY  2.419674e+01  -3.310963  16.510949  29.469443  10.666708  -2.356933   
PFF  1.321779e+01  -2.172484  16.535470  -1.297733  13.422438   3.826750   
QQQ  1.840974e+01   1.885569  15.891390  32.425402  20.123716   9.765462   
SPY  1.313734e+01   0.852352  14.170933  29.001466  14.561671   1.288525   
TIP  5.856141e+00  13.771906   6.433704  -8.437707   3.252518  -2.381569   
TLT  9.109792e+00  35.019646   3.970094 -12.207634  26.915635  -2.869510   
VNQ  2.893837e+01   6.651841  16.647781   1.044215  30.378809   0.912580   

                                                                           \
           2016         2017       2018       2019        2020       2021   
BIL    0.087546     0.645078   1.726218   2.043093    0.365647  -0.098322   
BND    2.572363     3.986560   0.399135   8.614494    7.573912  -1.727393   
BTC  121.889824  1318.015197 -72.595405  87.161745  302.791925  57.643512   
CWB   11.740552    15.850029  -3.390546  22.445526   52.075130   2.723797   
DBC   19.457011     6.134192 -12.053876  11.531357   -7.952418  42.134068   
EEM   13.984428    35.635649 -16.880274  17.921055   14.694396  -4.243974   
EFA    2.950518    24.300268 -14.453725  22.293151    6.521341  10.868402   
EMB    9.380360     9.905451  -5.714477  15.477057    5.292724  -1.685333   
GLD    6.531248    11.930841  -3.116262  17.777954   23.904136  -6.235943   
HYG   14.093977     5.634443  -2.113881  14.243622    4.029842   3.970508   
IWD   18.637108    12.643549  -8.864450  26.145958    2.320766  26.654628   
IWF    8.867242    28.979828  -2.603118  35.965307   36.369542  29.256412   
IWM   24.476101    14.016077 -11.952838  24.682053   19.951130  16.051363   
LQD    6.260694     6.963650  -3.141747  17.005288   10.736396  -1.341631   
MDY   22.009398    15.307886 -12.125377  26.231429   13.336701  26.181833   
PFF    0.833675     7.262156  -4.110806  14.853392    6.792507   8.186217   
QQQ    9.406123    31.487189  -1.846574  38.405266   45.968830  29.244694   
SPY   13.585779    20.781421  -5.247180  31.087488   17.235231  30.505467   
TIP    4.272554     2.820250  -1.182766   8.181727   10.298569   5.561339   
TLT    0.446881     8.717174  -0.537742  13.520500   16.832865  -4.484083   
VNQ    9.896947     4.556880  -5.674558  31.773467   -3.476631  45.257505   

                                            TOTAL                
          2022        2023       2024         AVG           CUM  
BIL   1.415661    4.933407   1.391984    0.811017     12.737969  
BND -12.523828    5.098448  -1.277693    2.273851     37.233712  
BTC -65.299639  154.225467  57.022853  178.898456  13516.850419  
CWB -20.519189   14.907344   1.114219    9.480801    233.940600  
DBC  18.882232   -3.894198   8.645926    1.854677      5.755335  
EEM -21.128769    8.036133  

### Column summary

In [83]:
positive_pct = lambda x: (x > 0).mean() * 100

dfrc = dfr.agg(['idxmax', 'idxmin', positive_pct])
dfrc.index = ['Highest', 'Lowest', 'Positive Years (%)']



In [85]:
ds.save_df(dfrc.round(2), 'etf_summary_rows', max_cols=4)

### Rename index

In [71]:
dfr.index = dfr.index.map(df_tickers['full'])

In [72]:
dft = pd.concat({
    'ASSET': dfr.sort_values(('TOTAL', 'CUM'), ascending=False),
    'TOTAL': dfrc
})

## Gradient

In [73]:
from mplcustom import preprocessing

In [74]:
gmap = dft.loc['ASSET', 'YEARLY'].apply(preprocessing.scale_numbers, axis=1, result_type='expand').values

In [75]:
dft

YEARLY                        \
                                           2010       2011       2012   
ASSET Bitcoin (BTC)                         NaN        NaN        NaN   
      US Nasdaq 100 (QQQ)             18.409738   1.885569   15.89139   
      US Growth (IWF)                 15.009214   1.383875   13.56242   
      US Large Caps (SPY)             13.137337   0.852352  14.170933   
      US Mid Caps (MDY)               24.196744  -3.310963  16.510949   
      US Value (IWD)                  13.268779  -0.983614  15.552523   
      US Small Caps (IWM)             23.863385  -6.059659   14.80463   
      US REITs (VNQ)                  28.938366   6.651841  16.647781   
      Convertible Bonds (CWB)         12.804183  -7.975849  13.982726   
      EAFE Stocks (EFA)                5.404694  -12.94791   15.60895   
      High Yield Bonds (HYG)          11.040991   6.671975  10.696739   
      Preferred Stocks (PFF)          13.217785  -2.172484   16.53547   
      Gold (GLD)                      26.338795  10.137685   3.912267   
      EM Bonds (USD) (EMB)            10.675017   7.198319  17.603877   
      Investment Grade Bonds (LQD)     8.742207   9.317315  10.331629   
      Long Duration Treasuries (TLT)   9.109792  35.019646   3.970094   
      TIPS (TIP)                       5.856141  13.771906   6.433704   
      US Total Bond Market (BND)       6.077207   8.064261   3.262274   
      EM Stocks (EEM)                 13.221947  -19.57026  15.517695   
      US Cash (BIL)                         0.0  -0.061076  -0.021803   
      Commodities (DBC)                9.152138  -2.999639   0.506525   
TOTAL Highest                               VNQ        TLT        EMB   
      Lowest                                BIL        EEM        BIL   
      Positive Years (%)              95.238095  52.380952   90.47619   

                                                                       \
                                           2013       2014       2015   
ASSET Bitcoin (BTC)                         NaN -29.987059  37.014595   
      US Nasdaq 100 (QQQ)             32.425402  20.123716   9.765462   
      US Growth (IWF)                 29.754817  13.778403   5.590898   
      US Large Caps (SPY)             29.001466  14.561671   1.288525   
      US Mid Caps (MDY)               29.469443  10.666708  -2.356933   
      US Value (IWD)                   28.93702  14.254313  -3.999696   
      US Small Caps (IWM)             34.733873   6.186893   -3.92473   
      US REITs (VNQ)                   1.044215  30.378809    0.91258   
      Convertible Bonds (CWB)         19.287528   8.005454  -0.838895   
      EAFE Stocks (EFA)                19.59344  -4.500392  -0.568004   
      High Yield Bonds (HYG)           4.765883   1.719011  -5.021535   
      Preferred Stocks (PFF)          -1.297733  13.422438    3.82675   
      Gold (GLD)                     -28.834955  -3.745761 -11.062415   
      EM Bonds (USD) (EMB)            -8.047667   6.025027   1.395162   
      Investment Grade Bonds (LQD)    -2.227873   8.001854  -1.601613   
      Long Duration Treasuries (TLT) -12.207634  26.915635   -2.86951   
      TIPS (TIP)                      -8.437707   3.252518  -2.381569   
      US Total Bond Market (BND)      -1.955761   5.725347   0.218439   
      EM Stocks (EEM)                 -5.502211  -0.048793 -15.050538   
      US Cash (BIL)                   -0.087317  -0.065566  -0.109297   
      Commodities (DBC)               -7.962695 -27.046266 -26.714207   
TOTAL Highest                               IWM        VNQ        BTC   
      Lowest                                GLD        BTC        DBC   
      Positive Years (%)              47.619048  71.428571  38.095238   

                                                                          \
                                            2016         2017       2018   
ASSET Bitcoin (BTC)                   121.889824  1318.015197 -72.595405   
      US Nasdaq

In [76]:
dfs = (dft
 .style
    .format("{:,.2f}", subset=pd.IndexSlice['ASSET',:])
    .format("{:.2f}", subset=pd.IndexSlice['TOTAL', 'Positive Years (%)',:])
    .background_gradient(cmap='RdYlGn', axis=None, gmap=gmap, subset=pd.IndexSlice['ASSET', 'YEARLY'])
    .highlight_null(props='color: transparent; background-color: transparent;')
)

dfs

In [77]:
from mplcustom import datastyle as ds

In [78]:
dfsc = ds.set_caption(
    df=dfs, title='Asset performance 2010-2024: a comparative analysis', subtitle='Evaluating returns across cryptocurrency, stocks, bonds, and commodities. Inspired in @charliebilello.',
    title_size=25,
)

In [79]:
ds.save_df(dfsc, 'asset_performance')